In [1]:
from operator import index
import pandas as pd
import numpy as np
import torch as T
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [2]:
def load_data(pth):

    data = pd.read_csv(pth)

    data.drop(['date'], axis = 1, inplace = True)

    y = data['DAX'].values
    x = data.drop(['DAX'], axis = 1).values

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

    x_train = T.tensor(x_train.astype(np.float32))
    x_test = T.tensor(x_test.astype(np.float32))

    y_train = T.tensor(y_train.astype(np.float32))
    y_test = T.tensor(y_test.astype(np.float32))

    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = load_data(r'C:\Users\rnr31\Documents\GitHub\Data_Science_2022\03.Deep Learning\03. MLP Regression\data\turkish_stocks.csv')

In [3]:
def to_batches(x_train, x_test, y_train, y_test, batch_size):

    n_batches = x_train.shape[0] // batch_size # 11 / 3 = 3.66 -> 3
    n_batches_test = x_test.shape[0] // batch_size

    indexes = np.random.permutation(x_train.shape[0])
    indexes_test = np.random.permutation(x_test.shape[0])


    x_train = x_train[indexes]
    y_train = y_train[indexes]

    x_test = x_test[indexes_test]
    y_test = y_test[indexes_test]

    x_train = x_train[ :batch_size * n_batches ].reshape(n_batches, batch_size, x_train.shape[1])
    y_train = y_train[ :batch_size * n_batches ].reshape(n_batches, batch_size, 1)
    
    x_test = x_test[ :batch_size * n_batches_test ].reshape(n_batches_test, batch_size, x_test.shape[1])
    y_test = y_test[ :batch_size * n_batches_test ].reshape(n_batches_test, batch_size, 1)


    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = to_batches(x_train, x_test, y_train, y_test, 10)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

torch.Size([42, 10, 8])
torch.Size([42, 10, 1])
torch.Size([10, 10, 8])
torch.Size([10, 10, 1])


In [4]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.input_layer = nn.Linear(8, 64)
        self.hidden1 = nn.Linear(64,32)
        self.hidden2 = nn.Linear(32,8)
        self.output_layer = nn.Linear(8,1)

    def forward(self, x):
        first_layer = self.input_layer(x)
        act1 = F.relu(first_layer)
        second_layer = self.hidden1(act1)
        act2 = F.relu(second_layer)
        third_layer = self.hidden2(act2)
        act3 = F.relu(third_layer)
        out_layer = self.output_layer(act3)
        #x = F.relu(out_layer)
        return out_layer

model = Regressor()
model

Regressor(
  (input_layer): Linear(in_features=8, out_features=64, bias=True)
  (hidden1): Linear(in_features=64, out_features=32, bias=True)
  (hidden2): Linear(in_features=32, out_features=8, bias=True)
  (output_layer): Linear(in_features=8, out_features=1, bias=True)
)

In [5]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [6]:
epochs = 40
print_every = 40
epoch_mean_loss = 0

for epoch in range(epochs):
    running_loss = 0
    loss_list = []
    print(f"Epoch: {epoch+1}/{epochs}")

    for i, features in enumerate(iter(x_train)):
        # print(i)
        # print(features.shape)
        # print(x_train.shape)
        
        optimizer.zero_grad()

        pred = model.forward(features)
        loss = criterion(pred, y_train[i])

        loss.backward()
        
        
        optimizer.step()

        loss_list.append(loss.item())
        running_loss += loss.item()

        if i%print_every ==0:
            print(f"\tIteration: {i}\t Loss: {running_loss/print_every:.10f}")
            running_loss=0

    total_sum = sum(loss_list)
    mean_loss = round((total_sum/len(loss_list)),2)
    # print(f'End of epoch {epoch}')
    # print(mean_loss)
    # print(epoch_mean_loss)
    # if mean_loss != epoch_mean_loss:
    #     epoch_mean_loss = mean_loss
    # else:
    #     break


Epoch: 1/40
	Iteration: 0	 Loss: 0.0004898156
	Iteration: 40	 Loss: 0.0205745548
Epoch: 2/40
	Iteration: 0	 Loss: 0.0003872030
	Iteration: 40	 Loss: 0.0165824392
Epoch: 3/40
	Iteration: 0	 Loss: 0.0003057395
	Iteration: 40	 Loss: 0.0133814742
Epoch: 4/40
	Iteration: 0	 Loss: 0.0002410886
	Iteration: 40	 Loss: 0.0108121255
Epoch: 5/40
	Iteration: 0	 Loss: 0.0001897834
	Iteration: 40	 Loss: 0.0087480018
Epoch: 6/40
	Iteration: 0	 Loss: 0.0001490901
	Iteration: 40	 Loss: 0.0070884924
Epoch: 7/40
	Iteration: 0	 Loss: 0.0001168426
	Iteration: 40	 Loss: 0.0057533742
Epoch: 8/40
	Iteration: 0	 Loss: 0.0000913253
	Iteration: 40	 Loss: 0.0046786271
Epoch: 9/40
	Iteration: 0	 Loss: 0.0000711682
	Iteration: 40	 Loss: 0.0038130638
Epoch: 10/40
	Iteration: 0	 Loss: 0.0000552792
	Iteration: 40	 Loss: 0.0031156553
Epoch: 11/40
	Iteration: 0	 Loss: 0.0000427868
	Iteration: 40	 Loss: 0.0025535162
Epoch: 12/40
	Iteration: 0	 Loss: 0.0000329998
	Iteration: 40	 Loss: 0.0021002654
Epoch: 13/40
	Iteration: 